# Useful Utilities

**Authors:** Olivia Lynn, Eric Charles, Sam Schmidt, Alex Malz

**Last Run Sucessfully:** August 3, 2023

import_and_attach_all

explain
note that it's a useful shortcut, but slower, and imports stuff you'll never need (but helpful for notebooks esp)
vs as you become more advanced user you can do more specific imports
also mention how they put the classes directly in namespace, so you don't have to do rail.creation.etc

In [1]:
import rail
import ceci
import rail.stages

from rail.core.utils import RAILDIR
rail.stages.import_and_attach_all()
from rail.stages import *


Imported rail.hub
Imported rail.astro_tools
Imported rail.core
Imported rail.stages
Imported rail.bpz
Imported rail.cmnn
Imported rail.delight
Failed to import rail.dsps because: You need to have the SPS_HOME environment variable
Imported rail.flexzboost
Failed to import rail.gpz because: No module named 'rail.estimation.algos.gpz_v1'
Imported rail.pipelines
Failed to import rail.pzflow because: No module named 'rail.estimation.algos.pzflow'
Imported rail.sklearn
Imported rail.som
Attached 10 base classes and 46 fully formed stages to rail.stages


In [2]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [3]:
# TODO - explain this, and how at first it'll be empty, but that we'll run it later and see
# this keeps track of (ASK or check) (the things we put in the data store? the file handles? all of the above?)
DS.keys()

dict_keys([])

In [4]:
## make a pipeline to use

# TODO - explain find_rail_file

# we could do:
flow_file = os.path.join(
    RAILDIR, "rail/examples_data/goldenspike_data/data/pretrained_flow.pkl"
)

# but if we don't know where our file is, or we're just kinda lazy:
from rail.core.utils import find_rail_file
flow_file = find_rail_file('examples_data/goldenspike_data/data/pretrained_flow.pkl')
#os.environ['FLOWDIR'] = os.path.dirname(flow_file) # this was in the block i copy/pasted, but maybe throw out?


In [5]:


bands = ["u", "g", "r", "i", "z", "y"]
band_dict = {band: f"mag_{band}_lsst" for band in bands}
rename_dict = {f"mag_{band}_lsst_err": f"mag_err_{band}_lsst" for band in bands}
post_grid = [float(x) for x in np.linspace(0.0, 5, 21)]


In [6]:
flow_engine_test = FlowCreator.make_stage(
    name="flow_engine_test", model=flow_file, n_samples=50
)
col_remapper_test = ColumnMapper.make_stage(
    name="col_remapper_test", hdf5_groupname="", columns=rename_dict
)
#flow_engine_test.sample(6, seed=0).data

pipe = ceci.Pipeline.interactive()
stages = [flow_engine_test, col_remapper_test]
for stage in stages:
    pipe.add_stage(stage)

Inserting handle into data store.  model: /Users/orl/code/DESC-RAIL/rail_base/src/rail/examples_data/goldenspike_data/data/pretrained_flow.pkl, flow_engine_test


In [7]:

col_remapper_test.connect_input(flow_engine_test)

Inserting handle into data store.  output_flow_engine_test: inprogress_output_flow_engine_test.pq, flow_engine_test


## Methods of introspection

In [8]:
DS.keys()

dict_keys(['model', 'output_flow_engine_test'])

### TODO add code + explanation of eric's introspection 

In [9]:
# Get the names of the stages
pipe.stage_names


['flow_engine_test', 'col_remapper_test']

In [10]:
# Get the configuration of a particular stage
pipe.flow_engine_test.config


StageConfig{output_mode:default,n_samples:50,seed:12345,name:flow_engine_test,model:/Users/orl/code/DESC-RAIL/rail_base/src/rail/examples_data/goldenspike_data/data/pretrained_flow.pkl,config:None,aliases:{'output': 'output_flow_engine_test'},}

In [11]:
# TODO - is this a logical place to mention this?
# Update a particular config value once the stage has been created
pipe.flow_engine_test.config.update(seed=42)
pipe.flow_engine_test.config

StageConfig{output_mode:default,n_samples:50,seed:42,name:flow_engine_test,model:/Users/orl/code/DESC-RAIL/rail_base/src/rail/examples_data/goldenspike_data/data/pretrained_flow.pkl,config:None,aliases:{'output': 'output_flow_engine_test'},}

In [12]:
# Get the list of outputs 'tags'
# These are how the stage thinks of the outputs, as a list names associated to DataHandle types.
pipe.flow_engine_test.outputs


[('output', rail.core.data.PqHandle)]

In [13]:
# Get the list of outputs 'aliased tags'
# These are how the pipeline things of the outputs, as a unique key that points to a particular file
pipe.flow_engine_test._outputs


{'output_flow_engine_test': 'output_flow_engine_test.pq'}

In [14]:
# TODO - talk about resume=True
# notes: adding resume=True to pipe.initialize skips stages that already have existing output files
pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=True), None
)

Skipping stage flow_engine_test because its outputs exist already


(({'col_remapper_test': <Job col_remapper_test>},
  [Stage that applies remaps the following column names in a pandas DataFrame:
   f{str(self.config.columns)}]),
 {'output_dir': '.', 'log_dir': '.', 'resume': True})

In [15]:
# TODO - run this after initializing the pipeline, should show order of classes / "all the stages this pipeline will run"
pipe.stages

 Stage that applies remaps the following column names in a pandas DataFrame:
 f{str(self.config.columns)}]

### TODO explain RailStage.pipeline_stages

note that this will only look at what stages you've imported, which is not necessarily all stages in rail

In [16]:
for val in RailStage.pipeline_stages.values():
    if issubclass(val[0], rail.estimation.estimator.CatEstimator):
        print(val[0])

<class 'rail.estimation.estimator.CatEstimator'>
<class 'rail.estimation.algos.random_gauss.RandomGaussEstimator'>
<class 'rail.estimation.algos.train_z.TrainZEstimator'>
<class 'rail.estimation.algos.bpz_lite.BPZliteEstimator'>
<class 'rail.estimation.algos.cmnn.CMNNPDF'>
<class 'rail.estimation.algos.flexzboost.FlexZBoostEstimator'>
<class 'rail.estimation.algos.k_nearneigh.KNearNeighEstimator'>
<class 'rail.estimation.algos.sklearn_neurnet.SklNeurNetEstimator'>
<class 'rail.estimation.algos.nz_dir.NZDirSummarizer'>


### TODO talk about dir() -> is there a better way to do this?

In [17]:
# notes: this includes (**ASK**) methods you can call and parameters you can set
dir(pipe)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_stage',
 'build_config',
 'build_dag',
 'build_stage',
 'callback',
 'create',
 'enqueue_job',
 'find_all_outputs',
 'get_stage_aliases',
 'global_config',
 'initialize',
 'initiate_run',
 'interactive',
 'launcher_config',
 'make_flow_chart',
 'modules',
 'ordered_stages',
 'overall_inputs',
 'pipeline_files',
 'pipeline_outputs',
 'print_stages',
 'read',
 'remove_stage',
 'run',
 'run_config',
 'run_info',
 'run_jobs',
 'save',
 'should_skip_stage',
 'sleep',
 'stage_config_data',
 'stage_execution_config',
 'stage_names',
 'stages',
 'stages_config']

## In Conclusion

In [18]:
# TODO - write this out
# locally modilfy your .gitattributes to keep from committing notebook output 
# https://stackoverflow.com/questions/28908319/how-to-clear-jupyter-notebooks-output-in-all-cells-from-the-linux-terminal